# Structured Optimization
## Loic Landrieu. Feb 2019

# 1 - Data and Library Loading

In [ ]:
import sklearn
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import random
from sklearn.cluster import KMeans
random.seed(1)

1/ Load and plot the training data. Which model seems to have generated the data? In particular, how many "centroids" can you count?

In [ ]:
data_train = np.genfromtxt('train.dat', delimiter=" ", dtype='f8')
data_test = np.genfromtxt('test.dat', delimiter=" ", dtype='f8')
n_point, n_dimension = data_train.shape
n_state = 4

In [ ]:
plt.scatter(data_train[:,0], data_train[:,1])
plt.show()

# 2 First approach: k-means algorithm

2/ Use the $k$-means algorithm in order to cluster the data into $K$ classes stored in vector $z\in\{0,1,2,3\}^T$. Return the the vector $\tau \in \{0,1\}^{T \times K}$ such that
$
\tau_{t,k} = 
\begin{cases}
1 & \text{if}\;\;z_t=k\\
0 & \text{otherwise}.
\end{cases}
$

Hint: use the $\texttt{sklearn}$ library.

In [ ]:
def kmeans(x, K):
    """
    inputs:
    x : [T,D], T points of dimension D 
    K : integer, number of clusters
    outputs:
    tau [T,k] output, one-hot encoding of the clustering in k classes
    """
    T = x.shape[0]
    kmeans = KMeans(n_clusters=K, random_state=0).fit(x)
    z = kmeans.labels_
    tau = np.zeros((T, K),dtype='f8')
    #FILL TAU
    return tau

3/ Compute and plot the k-means affectation for $K=4$. <font color=red> What assumption of the $k$-means algorithm is not well-suited to the data?

In [ ]:
def plot_affectation(x, cluster_indices):
    colors = ['k', 'r', 'g', 'b']
    markers = [ 'o', 's', '+', 'x']
    for i_state in range(n_state):
        plt.scatter(x[cluster_indices==i_state,0], x[cluster_indices==i_state,1], c=colors[i_state], marker=markers[i_state])

In [ ]:
tau=#TODO
plot_affectation(data_train, #TODO)

# 3 EM-algorithm for the Gaussian Mixture Model

We first model the data by a Gaussian Mixture Model (GMM): the observed data is comprised of $T$ i.i.d random variables $X_t \mapsto \mathbb{R}^{2}$  which follow a mixture of $K$ Gaussians. The choice of the Gaussian component in the mixture is determined by a latent variable $Z_t \mapsto \{0,\cdots,K-1\}$ following a multinomial distribution.
To each entry $t \in [0,\cdots,T-1]$ we associate an observation $x_t$ and a latent variable $z_t$, and denote by $x\in \mathbb{R}^{T \times 2}$ the vector of $T$ observations and $z\in \{0,\cdots,K-1\}^{T}$ a corresponding vector of latent variables.

$$
p(x,z) = \prod_{t=0}^{T-1} p(x_t,z_t) = \prod_{t=0}^{T-1} p(z_t) p(x_t \mid z_t)~.
$$

The latent variables follow each a multinomial distribution parameterized by $\pi \in \mathbb{R}^K$: 
$$
p(z_t=k) = \pi_k~.
$$
The conditional probability $p(x_t \mid z_t=k)$ follows a Gaussian distribution parameterized by the centroid $\mu_k \in \mathbb{R}^{2}$ and covariance  matrix $\Sigma_k \in \mathbb{R}^{ 2 \times 2}$:
$$
p(x_t\mid z_t=k) \sim \frac1{(2\pi)^{D/2} \mid\Sigma_i\mid^{1/2}}
\exp{\left(-\frac12(x_t-\mu_k)^\intercal \Sigma_k^{-1} (x_t-\mu_k)\right)},
$$

5 / Draw the model in plate notation.

6 / Complete the function $\texttt{log_gaussian}(x, mu, Sigma)$ which returns the logarithm of the emission probability of value $x$ by a normal distribution parameterized by $\mu$ and $\Sigma$. Run the test cell to check if your code is correct.

Hint: use `np.linalg.det` and `np.linalg.inv`

In [ ]:
def log_gaussian(x, mu, Sigma):
    """
    inputs:
    x : [T,D], input T points of dimension D
    mu : [D,], centroid
    Sigma : [D,D], centroid
    outputs:
    float, log probability of x being generated by the gaussian N(mu, sigma)
    """
    vec_diff = x - mu
    return #TODO

In [ ]:
#the next assert checks the correcteness of the log-gaussian function's code! the assert must pass
mu = np.array([1,1])
Sigma = np.array([[1,0.5],[0.3,2]])
x = np.array([[1,0],[10,1]])
assert((np.abs(log_gaussian(x, mu, Sigma)-np.array([ -2.41574016, -45.92925367]))<1e-8).all())

8 / Complete the function $\texttt{log_all_gaussians}(x, mus, Sigmas)$ which returns $p(x_t \mid z_t=k)$ for all the $k$ Gaussian distributions parameterized by $\mu_k$ and $\Sigma_k$:
$$
[p]_{t,k}=p(x_t \mid z_t=k)
$$

In [ ]:
def log_all_gaussians(x, mus, Sigmas):
    """
    inputs:
    x : [T,D] T input points of dimension D
    mus : [K,D] the K centroids
    Sigmas : [K,D,D] the K covariances
    outputs:
    [T,K]  probability that each data point x_t was generated by the k-th mixture
    """
    n_state = mus.shape[0]
    log_proba = np.zeros((x.shape[0], n_state),dtype='f8')
    for i_state in range(n_state):
        log_proba[:,i_state] = #TODO
    return log_proba

In [ ]:
#another code check
mus = np.array([[1,1],[5,5]])
Sigmas = np.array([[[1,0.5],[0.3,2]],[[2,0.5],[0.9,2]]])
x = np.array([[1,0],[10,1]])
assert((np.abs(log_all_gaussians(x, mus, Sigmas)-np.array([[ -2.41574016, -10.07698467], [-45.92925367, -17.96430861]]))<1e-7).all())

6 / Complete the function $\texttt{M_step_mixture}$ which implements the M-step of an EM algorithm for a GMM model. This function returns the parameters $\hat{\pi},\hat{\mu}, \hat{\Sigma}$ learnt from the expected latent variables $\tau_{t,k}=p(z_t=k \mid x; \theta_\text{current})$.

Reminder:
$$
\hat{\pi}_k = \frac{\sum_{t=0}^{T-1}\tau_{t,k}}{T} 
$$
$$
\hat{\mu}_k = \frac{\sum_{t=0}^{T-1}\tau_{t,k} x_k}{\sum_{t=0}^{T-1} \tau_{t,k}} 
$$
$$
\hat{\Sigma}_k = \frac{\sum_{t=0}^{T-1}\tau_{t,k} (x_t-\hat{\mu_k})(x_t-\hat{\mu_k})^\intercal}{\sum_{t=0}^{T-1} \tau_{t,k}} 
$$

In [ ]:
#M-step
def M_step_mixture(x, tau):
    """
    x : [T,D] T input points of dimension D
    tau : [T,K] expected latent variable
    outputs:
    pi : [K,] the Multinomial parameters of z
    mus : [K,D] the K centroids
    Sigmas : [K,D,D] the K covariances
    """
    pi = np.zeros((n_state,),dtype='f8')
    mus = np.zeros((n_state, n_dimension),dtype='f8')
    Sigmas = np.zeros((n_state, n_dimension, n_dimension),dtype='f8')
    tau_sum  = tau.sum(0)
    pi = #TODO
    mus = #TODO
    for k_state in range(n_state):
        x_diff = x - mus[k_state,:][None,:]
        Sigmas[k_state,:,:] = #TODO
    return pi, mus, Sigmas

5/ Compute $\pi, (\mu_k, \Sigma_k)_k, $ with the value of $\tau$ obtained with $k$-means. Use the function $\texttt{plot_model}(\tau, mus, Sigmas)$ to represent the iso-density contour of each Gaussians. <font color=red> Why does it make sens to use the $\tau$ from k-means with an M-step to compute the parameters of the Gaussians.

In [ ]:
def plot_model(x, tau, mus, Sigmas):
    def plot_gaussian(mu, Sigma):
        x, y = np.meshgrid(np.linspace(-10, 10, 100),
                       np.linspace(-10, 10, 100))
        def gauss(x,y): return np.exp(log_gaussian(np.array([x,y])[None,:], mu, Sigma)[0])
        vec_gaussian = np.vectorize(gauss)
        plt.contour(x, y, vec_gaussian(x, y),3)
    plt.clf()
    colors = ['k', 'r', 'g', 'b']
    markers = [ 'o', 's', '+', 'x']
    z = tau.argmax(1)
    for k_state in range(n_state):
        plt.scatter(x[z==k_state,0], x[z==k_state,1], c=colors[k_state], marker=markers[k_state])
        plot_gaussian(mus[k_state,:], Sigmas[k_state,:,:])

In [ ]:
tau=kmeans(data_train, 4)
pi, mus, Sigmas = M_step_mixture(data_train, tau)
plot_model(data_train, tau, mus, Sigmas)

7/ Complete the function $\texttt{logsumexp}(X)=\log(\sum_k(\exp(X_k)))$ which compute the logarithm of sum on the last dimension of elementwise exponentials of an array in a robust way. Try it on the example below, and explain why the naive approach doesn't work.

<font color=red> Justify why this might be necessary when handling normal distributions.

In [ ]:
def logsumexp(x):
    """
    #compute log(sum(exp(x))) avoiding numerical underflow
    """    
    max_per_line = x.max(-1, keepdims=True)
    return (max_per_line + np.log(np.sum(np.exp(x-max_per_line),-1, keepdims=True))).squeeze()

In [ ]:
test_value = np.array([-1000, -2000, -1001])
naive = np.log(np.sum(np.exp(test_value)))
robust = logsumexp(test_value)
print("naive = %f / robust = %f" % (naive, robust))

7 / Complete the function $\texttt{E_step_mixture}(x, \pi, mus, Sigmas)$ which returns the expected latent values $\tau_{t,k}=p(z_t=k \mid x: \theta_\text{current})$ given by the mixture parameterization $\pi, (\mu_k, \Sigma_k)_k$. <font color='red'> Prove that <font color='black'>:
$$
p(z_t=k \mid x; \theta_\text{current})=\frac{p(x_t \mid z_t=k; \theta_\text{current})\;p(z_t=k; \theta_\text{current})}{\sum_{k=0}^{K-1} p(x_t \mid z_t=k; \theta_\text{current})\;p(z_t=k; \theta_\text{current})}
$$

In [ ]:
def E_step_mixture(x, pi, mus, Sigmas):
    """
    inputs:
    x : [T,D] T input points of dimension D
    pi : [K,] the Multinomial parameters of z
    mus : [K,D] the K centroids
    Sigmas : [K,D,D] the K covariances
    outputs:
    tau : [T,K] expected latent variable
    """
    log_tau_unnormalized = #TODO
    return #TODO   

In [ ]:
#check that the taus are probabilities
tau = E_step_mixture(data_train, pi, mus, Sigmas)
assert((np.abs(tau.sum(1)-1)<1e-8).all())

8 / Complete the function $\texttt{avg_log_likelihood_mixture}(x)=\log(p(x;\theta_\text{current}))$ which returns the loglikelihood of the vector of observations  $x$ given a Gaussian mixture model parameterized by $\pi, (\mu_k, \Sigma_k)_k$ and normalized by the number of observations.
$$
L(x;\theta_\text{current})=\frac1T \sum_{t=0}^{T-1}\log(p(x_t;\theta_\text{current}))=\frac1T \sum_{t=0}^{T-1}\log\left(\sum_{k=0}^{K-1} p(x_t,z_t=k;\theta_\text{current})\right)
$$

In [ ]:
def avg_log_likelihood_mixture(x, pi, mus, Sigmas):
    """
    inputs:
    x : [T,D] T input points of dimension D
    pi : [K,] the Multinomial parameters of z
    mus : [K,D] the K centroids
    Sigmas : [K,D,D] the K covariances
    outputs:
    log-likelihood of x
    """
    log_tau_unnormalized = log_all_gaussians(x, mus, Sigmas) + np.log(pi[None,:])#actually did it for you, nothing to complete
    return (logsumexp(log_tau_unnormalized).sum()) / x.shape[0]

9 / Implement the EM algorithm to learn the mixture parameters $\pi, (\mu_k, \Sigma_k)_k$ from the training set with $K=4$. Initialize with $k$-means. Print the likelihood at each step, what do we observe?

In [ ]:
def EM_mixture(x):
    """
    inputs:
    x : [T,D] T input points of dimension D
    outputs:
    pi0 : [K,] initial Multinomial
    mus : [K,D] the K centroids
    Sigmas : [K,D,D] the K covariances
    tau : [T,K] expected latent variable
    """
    n_ite = 50
    #first E-step
    tau = kmeans(x, 4)
    #main loop
    for i_ite in range(n_ite):
        #M-step
        pi, mus, Sigmas = #TODO
        #E-step
        tau = #TODO
        #likelihood
        print("Mixture iteration %d  - loglikelihood = %1.3f" % (i_ite, #TODO))
    return pi, mus, Sigmas, tau

In [ ]:
pi, mus, Sigmas, tau = EM_mixture(data_train)
plot_model(data_train, tau, mus, Sigmas)
plt.show()

In [ ]:
print("Train loglikelihood = %f" % avg_log_likelihood_mixture(data_train, pi, mus, Sigmas))

9 / Compute the log-likelihood of the test data, visualize the data and comment.

In [ ]:
tau = E_step_mixture(data_test, pi, mus, Sigmas)
plot_model(data_test, tau, mus, Sigmas)
print("Test loglikelihood = %f" % avg_log_likelihood_mixture(data_test, pi, mus, Sigmas))

# 4 Hidden Markov Model

10 / We now take into account the temporal structure of the data with a chain-structured hidden Markov model with discrete latent variable $z\in\{0,\cdots,K-1\}^T$:
$$
p(x, z) = p(z_0)\prod_{t=0}^{T-1}  p(x_t \mid z_t)\prod_{t=1}^{T-1} p(z_t \mid z_{t-1}).
$$
We model $p(x_t \mid z_t = k) \sim \mathcal{N}(\mu_k, \Sigma_k)$ like in the previous model, and $p(z_{t+1}=k \mid z_{t}=l)=A_{k,l}$ and finally $p(z_0)$ with a multinomial distribution parameterized by $\pi_0$.

We initialize the Gaussian parameters $\mu_k$ and $\Sigma_k$ with the values $\hat{\mu_k}$ and $\hat{\Sigma_k}$ computed in the mixture model. We initalize $\pi_0$ and $A$ as follows:
$$
\pi_0=\hat{\pi}
$$

$$
[A]_{k,l}=\hat{\pi}_k
$$
Complete the next cell to initialize $A$ and $\pi_0$. <font color = red> How can we interpret this initialization?

In [ ]:
A = #TODO
pi0 = #TODO

10 / Complete the function $\texttt{alpha_beta}(x, pi,mus,Sigmas, A)$ which peforms the marginal inference of the model.

we have the following formulas for the alpha and beta recursions:
$$
\alpha(z_{t}) = p(x_0, \cdots, x_t, z_t)  = p(x_t \mid z_t)\sum_{z_{t-1}=0}^{K-1}\alpha(z_{t-1})\;p(z_{t} \mid z_{t-1})
$$
$$
\beta(z_{t}) = p(x_{t+1}, \cdots, x_T \mid z_t) = \sum_{z_{t+1}=0}^{K-1}\beta(z_{t+1})\;p(z_{t+1}\mid z_{t})\;p(x_t \mid z_{t})
$$

In [ ]:
def alpha_beta(x, pi0, mus, Sigmas, A):
    """
    inputs:
    x : [T,D] T input points of dimension D
    pi0 : [K,] initial Multinomial
    mus : [K,D] the K centroids
    Sigmas : [K,D,D] the K covariances
    A : [K,K] transition probability
    outputs:
    log_proba_emission : [T,K] the probability that each data point x_t was generated by the k-th mixture
    alpha, beta : [T,K] 
    """
    T = x.shape[0]
    log_proba_emission = log_all_gaussians(x, mus, Sigmas)

    #forward pass
    log_alpha = np.zeros((T, n_state), dtype=('f8'))
    log_alpha[0,:] = #TODO
    for t in range(1,T):
        log_alpha[t,] = #TODO
    #backward pass
    log_beta = np.zeros((T, n_state), dtype=('f8'))  
    for t in range(T-2,-1,-1):
        log_beta[t,] = #TODO
    return log_proba_emission, log_alpha, log_beta

In [ ]:
log_proba_emission, log_alpha, log_beta = alpha_beta(data_train, pi0, mus, Sigmas, A)

11 / Compute the first order marginal probability $\gamma_t= {p(z_t \mid x)}$. Check that the normalization factor is the same for all $t$. Why is that the case? 
Reminder:
$$
\gamma(z_t)=p(z_t \mid x) \propto {\alpha(z_t)\beta(z_t)}
$$

In [ ]:
#marginal inference - first order
log_gamma_unnormalized = #TODO
normalizing_factor = #TODO
log_gamma = #TODO
#check that the normalizing factor is indeed constant
assert((np.abs(normalizing_factor.std())<1e-8).all())

Plot the evolution of the most likely state for each of the first 100 point and comment. <font color = red> How good was our initialization of $A$?

In [ ]:
z = #TODO
figure(figsize=(12,4))
plt.plot(z[0:100])
plt.show()

12 / Compute the pairwise marginals $\xi(z_{t-1},z_t)=p(z_{t-1},z_{t} \mid x; \theta_\text{current})$.

Reminder:
$$
\xi(z_{t-1},z_t)= \frac{\alpha(z_{t-1}) p(z_t \mid X; \theta_\text{current}) p(z_t \mid z_{t-1}; \theta_\text{current}) \beta(z_t)}{p(x; \theta_\text{current})}
$$

In [ ]:
log_xi = #TODO
#code check: the pair marginals are indeed probabilities
assert((np.abs(np.exp(log_xi).sum(1).sum(1)-1)<1e-10).all())

13 / Complete the function $\texttt{E_step_HMM}(pi,mus,Sigmas, A)$ which returns the singleton and pairwise marginals as well as the averaged loglikelihood of the observation.

In [ ]:
def E_step_HMM(x, pi0, mus,Sigmas, A):
    """
    inputs:
    x : [T,D] T input points of dimension D
    pi0 : [K,] initial Multinomial
    mus : [K,D] the K centroids
    Sigmas : [K,D,D] the K covariances
    A : [K,K] transition probability
    outputs:
    log_gamma : [T,K] marginals
    log_xi : [T, K, K] patwise marginals
    avg likelihood, float
    """
    #
    log_proba_emission, log_alpha, log_beta = #TODO
    log_gamma_unnormalized = #TODO

    log_gamma = #TODO
    log_xi = #TODO
    return log_gamma, log_xi, log_likelihood[0]/x.shape[0]

14 / Complete the function $\texttt{M_step_HMM}(\gamma, \xi)$ for the HMM model which returns the parameters $\hat{\mu}_k,\hat{\Sigma}_k, \hat{A}$.

$$
[\hat{A}]_{k,l}=\frac{\sum_{t=0}^{T-2}p(z_{t+1}=k,z_{t}=l \mid x; \theta_\text{current})}{\sum_{t=0}^{T-2}\sum_{j=0}^{K-1} p(z_{t+1}=j,z_{t}=l \mid x; \theta_\text{current})}
$$

In [ ]:
def M_step_HMM(x, log_gamma, log_xi):
    """
    inputs:
    x: [T,D] T input points of dimension D
    log_gamma [T,K] marginals
    log_xi [T, K, K] patwise marginals
    outputs:
    pi0 : [K,] initial Multinomial
    mus : [K,D] the K centroids
    Sigmas : [K,D,D] the K covariances
    A : [K,K] transition probability
    """
    pi, mus, Sigmas = #TODO
    A = #TODO
    pi0 = np.exp(log_gamma[0,:])
    return pi0, mus, Sigmas, A

In [ ]:
pi0, mus, Sigmas, A =  M_step_HMM(data_train, log_gamma, log_xi)
#a code check that A is indeed a conditional probability
assert(np.all(np.abs(A.sum(0)-1)<1e-10))

16 / Implement the EM algorithm for the HMM model. Initialize the parameters with EM on the Gaussian mixture model.

In [ ]:
#init with mixture
pi, mus, Sigmas, tau = EM_mixture(data_train)
A = np.tile(pi, (n_state,1))
pi0 = pi
n_ite = 100
#main loop
for i_ite in range(100):    
    #E-step
    log_gamma, log_xi, log_likelihood = #TODO
    print("HMM iteration %d  - likelihood = %1.3f" % (i_ite, log_likelihood))
    #M-step
    pi0, mus, Sigmas, A = #TODO
    
plot_model(data_train, np.exp(log_gamma), mus, Sigmas)
plt.show()

18/ Compute the final loglikelihood on the training set and compare it to the one obtained with the mixture model. <font color='red'> Is it a surprising result? Explain why? Make the same comparison on the test set. What conclusion can we draw now?</font>

In [ ]:
_log_gamma, _log_xi, log_likelihood_train = E_step_HMM(data_train, pi0, mus, Sigmas, A)
_log_gamma, _log_xi, log_likelihood_test = E_step_HMM(data_test, pi0, mus, Sigmas, A)
print("likelihood train %1.3f  - likelihood test= %1.3f" % (log_likelihood_train, log_likelihood_test))

19/ <font color = red> If we didn't know in advance the number of states, how would we choose it?

20 / Implement the Viterbi algorithm for MAP inference / decoding. <font color = red> Explain the difference between marginal and MAP inference.
</font>

Reminder:
the Viterbi algorithm is also called max-product. It computes the sequence of state with the maximum likelihood. It defines $p^\text{map}_{t,i}$ as the probability that the most probable path $0\cdots,t$ end in state $i$. We have the following recursion:
$$
p^\text{map}_{t,i} = p(x_t,z_t=i;\theta_\text{current}) \;\max_j( p^\text{map}_{t-1,j}\; p(z_t=i \mid z_{t-1}=j;\theta_\text{current})
$$
Once $p^\text{map}_{T,i}$ is computed, the last state of the map sequence is $\text{argmax}(p^\text{map}_{T,i})$. To compute the previous state, one simply backtrack through the selected transition computed earlier (keep track of the argmax!).

In [ ]:
def viterbi(x, pi0, mus, Sigmas, A):
    """
    inputs:
    pi0 : [K,] initial Multinomial
    mus : [K,D] the K centroids
    Sigmas : [K,D,D] the K covariances
    A : [K,K] transition probability
    outputs:
    map_state T, the MAP affectation
    best_previous_state : the argmax of each max-product
    """
    T = x.shape[0]
    log_proba_emission = #TODO
    map_state = np.zeros((T,), dtype='uint8') #the map sequence
    best_proba = np.zeros((T,n_state), dtype='f8') #probability of best sequence 1..t being in state i
    best_previous_state = np.zeros((T,n_state), dtype='uint8') #state of the previous observation in the best sequence 1..t
    #init
    best_proba[0,:] = #TODO
    best_previous_state[0,:] = np.NaN #no previous observation
    #forward pass
    for t in range(1,T):
        proba_previous_state = #TODO
        best_proba[t,] = #TODO
        best_previous_state[t,] = np.argmax(proba_previous_state,1)
    #backward pass
    map_state[-1] = best_proba[-1,:].argmax()
    for t in range(T-2,0,-1):#backtrack
        map_state[t] = #TODO
    return map_state, best_previous_state

In [ ]:
map_state, best_previous_state = viterbi(data_train, pi0, mus,Sigmas, A)
plot_affectation(data_train, map_state)

21 / Launch the visualiziation below representing the backtracking of the Viterbi algorithm. Interpret and comment.

In [ ]:
figure(num=None, figsize=(8, 100), dpi=80, facecolor='w', edgecolor='k')
plt.axis('off')
for t in range(n_point-2,n_point-100,-1):
    for i_state in range(n_state):
        if best_previous_state[t,i_state]==map_state[t-1] and i_state==map_state[t]:
            c = 'r'
        else:
            c = 'k'
        plt.plot([i_state, best_previous_state[t,i_state]],[t, t-1], c)
best_previous_state[-10:-1,:] 
plt.show()